In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

In [2]:
bulvar = gpd.read_file("C:/Users/malsh/Desktop/Магазин Магазинов/Бульварное.geojson")
sadovoe = gpd.read_file("C:/Users/malsh/Desktop/Магазин Магазинов/Садовое.geojson")
ttk = gpd.read_file("C:/Users/malsh/Desktop/Магазин Магазинов/ТТК.geojson")
mkad = gpd.read_file("C:/Users/malsh/Desktop/Магазин Магазинов/МКАД.geojson")
mkad_out = gpd.read_file("C:/Users/malsh/Desktop/Магазин Магазинов/За Мкадом.geojson")
novaya = gpd.read_file("C:/Users/malsh/Desktop/Магазин Магазинов/Новая Москва.geojson")

# Функция для определения района на основе координат
def get_district(point):
    if bulvar.geometry.contains(point).any():
        return 'Бульварное кольцо'
    elif sadovoe.geometry.contains(point).any():
        return 'Садовое кольцо'
    elif ttk.geometry.contains(point).any():
        return 'ТТК'
    elif mkad.geometry.contains(point).any():
        return 'МКАД'
    elif mkad_out.geometry.contains(point).any():
        return 'За МКАДом'
    elif novaya.geometry.contains(point).any():
        return 'Новая Москва'
    else:
        return 'Неизвестно'

In [3]:
def categorize_area(area):
    if area < 100:
        return 'До 100 кв.м.'
    elif 100 <= area < 200:
        return '100-200 кв.м.'
    elif 200 <= area < 300:
        return '200-300 кв.м.'
    elif 300 <= area < 500:
        return '300-500 кв.м.'
    else:
        return 'Более 500 кв.м.'

In [4]:
def categorize_price(price):
    if price < 30000000:
        return 'До 30 млн'
    elif 30000000 <= price < 100000000:
        return '30-100 млн'
    elif 100000000 <= price < 500000000:
        return '100-500 млн'
    elif 500000000 <= price < 1000000000:
        return '500 млн-1 млрд'
    elif 1000000000 <= price < 3000000000:
        return '1-3 млрд'
    else:
        return 'Более 3 млрд'

## Октябрь 2021

In [5]:
df_oct_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2021/Октябрь_2021_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_oct_1 = df_oct_1.drop_duplicates()
#display(df_oct_1.head(3))
display(df_oct_1.shape)

df_oct_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2021/Октябрь_2021_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_oct_2 = df_oct_2.drop_duplicates()
#display(df_oct_2.head(3))
display(df_oct_2.shape)

df_oct_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2021/Октябрь_2021_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_oct_3 = df_oct_3.drop_duplicates()
#display(df_oct_3.head(3))
display(df_oct_3.shape)

df_oct_2021 = pd.concat([df_oct_1, df_oct_2, df_oct_3], ignore_index=True)
display(df_oct_2021.shape)

df_oct_2021 = df_oct_2021[df_oct_2021['Этаж'] == 1]
df_oct_2021 = df_oct_2021.drop_duplicates()
df_oct_2021 = df_oct_2021.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_oct_2021.shape)

df_oct_2021 = df_oct_2021[(df_oct_2021['Площадь, м2'] < 1000) & (df_oct_2021['Площадь, м2'] > 20)]
display(df_oct_2021.shape)

df_oct_2021 = df_oct_2021.dropna(subset=['Описание'])
condition1 = df_oct_2021['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_oct_2021['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_oct_2021 = df_oct_2021[~condition1 & ~mask]
display(df_oct_2021.shape)

df_oct_2021 = df_oct_2021[~(((df_oct_2021['Тип'] == 'Сдам') & (df_oct_2021['Стоимость, м2'] > 100000)) | ((df_oct_2021['Тип'] == 'Продам') & (df_oct_2021['Стоимость, м2'] > 10000000)))]
display(df_oct_2021.shape)

# Добавляем столбец "Категория Площади"
df_oct_2021['Категория Площади'] = df_oct_2021['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_oct_2021['Район'] = df_oct_2021.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_oct_2021['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_oct_2021.loc[df_oct_2021['Тип'] == 'Продам', 'Категория Стоимости'] = df_oct_2021.loc[df_oct_2021['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_oct_2021.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_oct_21_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_oct_21_street.xlsx"}')

(10000, 13)

(10000, 13)

(7287, 13)

(27287, 13)

(9578, 13)

(8471, 13)

(8278, 13)

(8255, 13)

C:\Users\malsh\anaconda3\lib\site-packages\shapely\predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_oct_21_street.xlsx


## Ноябрь 2021

In [6]:
df_nov_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2021/Ноябрь_2021_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_nov_1 = df_nov_1.drop_duplicates()
#display(df_nov_1.head(3))
display(df_nov_1.shape)

df_nov_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2021/Ноябрь_2021_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_nov_2 = df_nov_2.drop_duplicates()
#display(df_nov_2.head(3))
display(df_nov_2.shape)

df_nov_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2021/Ноябрь_2021_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_nov_3 = df_nov_3.drop_duplicates()
#display(df_nov_3.head(3))
display(df_nov_3.shape)

df_nov_2021 = pd.concat([df_nov_1, df_nov_2, df_nov_3], ignore_index=True)
display(df_nov_2021.shape)

df_nov_2021 = df_nov_2021[df_nov_2021['Этаж'] == 1]
df_nov_2021 = df_nov_2021.drop_duplicates()
df_nov_2021 = df_nov_2021.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_nov_2021.shape)

df_nov_2021 = df_nov_2021[(df_nov_2021['Площадь, м2'] < 1000) & (df_nov_2021['Площадь, м2'] > 20)]
display(df_nov_2021.shape)

df_nov_2021 = df_nov_2021.dropna(subset=['Описание'])
condition1 = df_nov_2021['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_nov_2021['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_nov_2021 = df_nov_2021[~condition1 & ~mask]
display(df_nov_2021.shape)

df_nov_2021 = df_nov_2021[~(((df_nov_2021['Тип'] == 'Сдам') & (df_nov_2021['Стоимость, м2'] > 100000)) | ((df_nov_2021['Тип'] == 'Продам') & (df_nov_2021['Стоимость, м2'] > 10000000)))]
display(df_nov_2021.shape)

# Добавляем столбец "Категория Площади"
df_nov_2021['Категория Площади'] = df_nov_2021['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_nov_2021['Район'] = df_nov_2021.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_nov_2021['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_nov_2021.loc[df_nov_2021['Тип'] == 'Продам', 'Категория Стоимости'] = df_nov_2021.loc[df_nov_2021['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_nov_2021.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_21_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_21_street.xlsx"}')

(10000, 13)

(10000, 13)

(4665, 13)

(24665, 13)

(5419, 13)

(4883, 13)

(4781, 13)

(4763, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_21_street.xlsx


## Декабрь 2021

In [7]:
df_dec_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Декабрь_2021/Декабрь_2021_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_1 = df_dec_1.drop_duplicates()
#display(df_dec_1.head(3))
display(df_dec_1.shape)

df_dec_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Декабрь_2021/Декабрь_2021_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_2 = df_dec_2.drop_duplicates()
#display(df_dec_2.head(3))
display(df_dec_2.shape)


df_dec_2021 = pd.concat([df_dec_1, df_dec_2], ignore_index=True)
display(df_dec_2021.shape)

df_dec_2021 = df_dec_2021[df_dec_2021['Этаж'] == 1]
df_dec_2021 = df_dec_2021.drop_duplicates()
df_dec_2021 = df_dec_2021.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_dec_2021.shape)

df_dec_2021 = df_dec_2021[(df_dec_2021['Площадь, м2'] < 1000) & (df_dec_2021['Площадь, м2'] > 20)]
display(df_dec_2021.shape)

df_dec_2021 = df_dec_2021.dropna(subset=['Описание'])
condition1 = df_dec_2021['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_dec_2021['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_dec_2021 = df_dec_2021[~condition1 & ~mask]
display(df_dec_2021.shape)

df_dec_2021 = df_dec_2021[~(((df_dec_2021['Тип'] == 'Сдам') & (df_dec_2021['Стоимость, м2'] > 100000)) | ((df_dec_2021['Тип'] == 'Продам') & (df_dec_2021['Стоимость, м2'] > 10000000)))]
display(df_dec_2021.shape)

# Добавляем столбец "Категория Площади"
df_dec_2021['Категория Площади'] = df_dec_2021['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_dec_2021['Район'] = df_dec_2021.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_dec_2021['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_dec_2021.loc[df_dec_2021['Тип'] == 'Продам', 'Категория Стоимости'] = df_dec_2021.loc[df_dec_2021['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_dec_2021.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_21_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_21_street.xlsx"}')

(10000, 13)

(6180, 13)

(16180, 13)

(4591, 13)

(4050, 13)

(3986, 13)

(3962, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_21_street.xlsx


In [25]:
three_m = pd.concat([df_oct_2021, df_nov_2021, df_dec_2021])
three_m.drop_duplicates()
three_m.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first', inplace=True)
display(three_m.shape)

three_m.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/4кв. 2021.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/4кв. 2021.xlsx"}')

(14773, 16)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/4кв. 2021.xlsx


## Январь 2022

In [8]:
df_jan_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2022/Январь_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_1 = df_jan_1.drop_duplicates()
#display(df_jan_1.head(3))
display(df_jan_1.shape)

df_jan_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2022/Январь_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_2 = df_jan_2.drop_duplicates()
#display(df_jan_2.head(3))
display(df_jan_2.shape)


df_jan_2022 = pd.concat([df_jan_1, df_jan_2], ignore_index=True)
display(df_jan_2022.shape)

df_jan_2022 = df_jan_2022[df_jan_2022['Этаж'] == 1]
df_jan_2022 = df_jan_2022.drop_duplicates()
df_jan_2022 = df_jan_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_jan_2022.shape)

df_jan_2022 = df_jan_2022[(df_jan_2022['Площадь, м2'] < 1000) & (df_jan_2022['Площадь, м2'] > 20)]
display(df_jan_2022.shape)

df_jan_2022 = df_jan_2022.dropna(subset=['Описание'])
condition1 = df_jan_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_jan_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_jan_2022 = df_jan_2022[~condition1 & ~mask]
display(df_jan_2022.shape)

df_jan_2022 = df_jan_2022[~(((df_jan_2022['Тип'] == 'Сдам') & (df_jan_2022['Стоимость, м2'] > 100000)) | ((df_jan_2022['Тип'] == 'Продам') & (df_jan_2022['Стоимость, м2'] > 10000000)))]
display(df_jan_2022.shape)

# Добавляем столбец "Категория Площади"
df_jan_2022['Категория Площади'] = df_jan_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_jan_2022['Район'] = df_jan_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_jan_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_jan_2022.loc[df_jan_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_jan_2022.loc[df_jan_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_jan_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_22_street.xlsx"}')

(10000, 13)

(9107, 13)

(19107, 13)

(6621, 13)

(5863, 13)

(5764, 13)

(5746, 13)

C:\Users\malsh\anaconda3\lib\site-packages\shapely\predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_22_street.xlsx


## Февраль 2022

In [9]:
df_feb_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2022/Февраль_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_1 = df_feb_1.drop_duplicates()
#display(df_feb_1.head(3))
display(df_feb_1.shape)

df_feb_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2022/Февраль_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_2 = df_feb_2.drop_duplicates()
#display(df_feb_2.head(3))
display(df_feb_2.shape)

df_feb_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2022/Февраль_2022_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_3 = df_feb_3.drop_duplicates()
#display(df_feb_3.head(3))
display(df_feb_3.shape)

df_feb_2022 = pd.concat([df_feb_1, df_feb_2, df_feb_3], ignore_index=True)
display(df_feb_2022.shape)

df_feb_2022 = df_feb_2022[df_feb_2022['Этаж'] == 1]
df_feb_2022 = df_feb_2022.drop_duplicates()
df_feb_2022 = df_feb_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_feb_2022.shape)

df_feb_2022 = df_feb_2022[(df_feb_2022['Площадь, м2'] < 1000) & (df_feb_2022['Площадь, м2'] > 20)]
display(df_feb_2022.shape)

df_feb_2022 = df_feb_2022.dropna(subset=['Описание'])
condition1 = df_feb_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_feb_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_feb_2022 = df_feb_2022[~condition1 & ~mask]
display(df_feb_2022.shape)

df_feb_2022 = df_feb_2022[~(((df_feb_2022['Тип'] == 'Сдам') & (df_feb_2022['Стоимость, м2'] > 100000)) | ((df_feb_2022['Тип'] == 'Продам') & (df_feb_2022['Стоимость, м2'] > 10000000)))]
display(df_feb_2022.shape)

# Добавляем столбец "Категория Площади"
df_feb_2022['Категория Площади'] = df_feb_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_feb_2022['Район'] = df_feb_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_feb_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_feb_2022.loc[df_feb_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_feb_2022.loc[df_feb_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_feb_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_22_street.xlsx"}')

(10000, 13)

(10000, 13)

(2776, 13)

(22776, 13)

(6797, 13)

(5876, 13)

(5758, 13)

(5731, 13)

C:\Users\malsh\anaconda3\lib\site-packages\shapely\predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_22_street.xlsx


## Март 2022

In [10]:
df_mar_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Март_2022/Март_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_mar_1 = df_mar_1.drop_duplicates()
#display(df_mar_1.head(3))
display(df_mar_1.shape)

df_mar_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Март_2022/Март_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_mar_2 = df_mar_2.drop_duplicates()
#display(df_mar_2.head(3))
display(df_mar_2.shape)

df_mar_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Март_2022/Март_2022_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_mar_3 = df_mar_3.drop_duplicates()
#display(df_mar_3.head(3))
display(df_mar_3.shape)

df_mar_2022 = pd.concat([df_mar_1, df_mar_2, df_mar_3], ignore_index=True)
display(df_mar_2022.shape)

df_mar_2022 = df_mar_2022[df_mar_2022['Этаж'] == 1]
df_mar_2022 = df_mar_2022.drop_duplicates()
df_mar_2022 = df_mar_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_mar_2022.shape)

df_mar_2022 = df_mar_2022[(df_mar_2022['Площадь, м2'] < 1000) & (df_mar_2022['Площадь, м2'] > 20)]
display(df_mar_2022.shape)

df_mar_2022 = df_mar_2022.dropna(subset=['Описание'])
condition1 = df_mar_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_mar_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_mar_2022 = df_mar_2022[~condition1 & ~mask]
display(df_mar_2022.shape)

df_mar_2022 = df_mar_2022[~(((df_mar_2022['Тип'] == 'Сдам') & (df_mar_2022['Стоимость, м2'] > 100000)) | ((df_mar_2022['Тип'] == 'Продам') & (df_mar_2022['Стоимость, м2'] > 10000000)))]
display(df_mar_2022.shape)

# Добавляем столбец "Категория Площади"
df_mar_2022['Категория Площади'] = df_mar_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_mar_2022['Район'] = df_mar_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_mar_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_mar_2022.loc[df_mar_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_mar_2022.loc[df_mar_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_mar_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_22_street.xlsx"}')

(10000, 13)

(10000, 13)

(5406, 13)

(25406, 13)

(9262, 13)

(8232, 13)

(8027, 13)

(7992, 13)

C:\Users\malsh\anaconda3\lib\site-packages\shapely\predicates.py:540: RuntimeWarning: invalid value encountered in contains
  return lib.contains(a, b, **kwargs)


DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_22_street.xlsx


In [26]:
three_m = pd.concat([df_jan_2022, df_feb_2022, df_mar_2022])
three_m.drop_duplicates()
three_m.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first', inplace=True)
display(three_m.shape)

three_m.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/1кв. 2022.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/1кв. 2022.xlsx"}')

(16662, 16)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/1кв. 2022.xlsx


## Апрель 2022

In [11]:
df_apr_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Апрель_2022/Апрель_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_apr_1 = df_apr_1.drop_duplicates()
#display(df_apr_1.head(3))
display(df_apr_1.shape)

df_apr_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Апрель_2022/Апрель_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_apr_2 = df_apr_2.drop_duplicates()
#display(df_apr_2.head(3))
display(df_apr_2.shape)


df_apr_2022 = pd.concat([df_apr_1, df_apr_2], ignore_index=True)
display(df_apr_2022.shape)

df_apr_2022 = df_apr_2022[df_apr_2022['Этаж'] == 1]
df_apr_2022 = df_apr_2022.drop_duplicates()
df_apr_2022 = df_apr_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_apr_2022.shape)

df_apr_2022 = df_apr_2022[(df_apr_2022['Площадь, м2'] < 1000) & (df_apr_2022['Площадь, м2'] > 20)]
display(df_apr_2022.shape)

df_apr_2022 = df_apr_2022.dropna(subset=['Описание'])
condition1 = df_apr_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_apr_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_apr_2022 = df_apr_2022[~condition1 & ~mask]
display(df_apr_2022.shape)

df_apr_2022 = df_apr_2022[~(((df_apr_2022['Тип'] == 'Сдам') & (df_apr_2022['Стоимость, м2'] > 100000)) | ((df_apr_2022['Тип'] == 'Продам') & (df_apr_2022['Стоимость, м2'] > 10000000)))]
display(df_apr_2022.shape)

# Добавляем столбец "Категория Площади"
df_apr_2022['Категория Площади'] = df_apr_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_apr_2022['Район'] = df_apr_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_apr_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_apr_2022.loc[df_apr_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_apr_2022.loc[df_apr_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_apr_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_22_street.xlsx"}')

(10000, 13)

(7616, 13)

(17616, 13)

(7521, 13)

(6607, 13)

(6484, 13)

(6446, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_22_street.xlsx


## Май 2022

In [12]:
df_may_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Май_2022/Май_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_may_1 = df_may_1.drop_duplicates()
#display(df_may_1.head(3))
display(df_may_1.shape)

df_may_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Май_2022/Май_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_may_2 = df_may_2.drop_duplicates()
#display(df_may_2.head(3))
display(df_may_2.shape)


df_may_2022 = pd.concat([df_may_1, df_may_2], ignore_index=True)
display(df_may_2022.shape)

df_may_2022 = df_may_2022[df_may_2022['Этаж'] == 1]
df_may_2022 = df_may_2022.drop_duplicates()
df_may_2022 = df_may_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_may_2022.shape)

df_may_2022 = df_may_2022[(df_may_2022['Площадь, м2'] < 1000) & (df_may_2022['Площадь, м2'] > 20)]
display(df_may_2022.shape)

df_may_2022 = df_may_2022.dropna(subset=['Описание'])
condition1 = df_may_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_may_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_may_2022 = df_may_2022[~condition1 & ~mask]
display(df_may_2022.shape)

df_may_2022 = df_may_2022[~(((df_may_2022['Тип'] == 'Сдам') & (df_may_2022['Стоимость, м2'] > 100000)) | ((df_may_2022['Тип'] == 'Продам') & (df_may_2022['Стоимость, м2'] > 10000000)))]
display(df_may_2022.shape)

# Добавляем столбец "Категория Площади"
df_may_2022['Категория Площади'] = df_may_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_may_2022['Район'] = df_may_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_may_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_may_2022.loc[df_may_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_may_2022.loc[df_may_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_may_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_22_street.xlsx"}')


(10000, 13)

(8375, 13)

(18375, 13)

(7673, 13)

(6761, 13)

(6626, 13)

(6601, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_22_street.xlsx


## Июнь 2022

In [13]:
df_jun_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Июнь_2022/Июнь_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jun_1 = df_jun_1.drop_duplicates()
#display(df_jun_1.head(3))
display(df_jun_1.shape)

df_jun_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Июнь_2022/Июнь_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jun_2 = df_jun_2.drop_duplicates()
#display(df_jun_2.head(3))
display(df_jun_2.shape)


df_jun_2022 = pd.concat([df_jun_1, df_jun_2], ignore_index=True)
display(df_jun_2022.shape)

df_jun_2022 = df_jun_2022[df_jun_2022['Этаж'] == 1]
df_jun_2022 = df_jun_2022.drop_duplicates()
df_jun_2022 = df_jun_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_jun_2022.shape)

df_jun_2022 = df_jun_2022[(df_jun_2022['Площадь, м2'] < 1000) & (df_jun_2022['Площадь, м2'] > 20)]
display(df_jun_2022.shape)

df_jun_2022 = df_jun_2022.dropna(subset=['Описание'])
condition1 = df_jun_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_jun_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_jun_2022 = df_jun_2022[~condition1 & ~mask]
display(df_jun_2022.shape)

df_jun_2022 = df_jun_2022[~(((df_jun_2022['Тип'] == 'Сдам') & (df_jun_2022['Стоимость, м2'] > 100000)) | ((df_jun_2022['Тип'] == 'Продам') & (df_jun_2022['Стоимость, м2'] > 10000000)))]
display(df_jun_2022.shape)

# Добавляем столбец "Категория Площади"
df_jun_2022['Категория Площади'] = df_jun_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_jun_2022['Район'] = df_jun_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_jun_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_jun_2022.loc[df_jun_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_jun_2022.loc[df_jun_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_jun_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jun_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jun_22_street.xlsx"}')

(10000, 13)

(9462, 13)

(19462, 13)

(7453, 13)

(6573, 13)

(6379, 13)

(6359, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jun_22_street.xlsx


In [27]:
three_m = pd.concat([df_apr_2022, df_may_2022, df_jun_2022])
three_m.drop_duplicates()
three_m.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first', inplace=True)
display(three_m.shape)

three_m.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/2кв. 2022.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/2кв. 2022.xlsx"}')

(16786, 16)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/2кв. 2022.xlsx


## Июль 2022

In [14]:
df_jul_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Июль_2022/Июль_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jul_1 = df_jul_1.drop_duplicates()
#display(df_jul_1.head(3))
display(df_jul_1.shape)

df_jul_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Июль_2022/Июль_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jul_2 = df_jul_2.drop_duplicates()
#display(df_jul_2.head(3))
display(df_jul_2.shape)

df_jul_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Июль_2022/Июль_2022_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jul_3 = df_jul_3.drop_duplicates()
#display(df_jul_3.head(3))
display(df_jul_3.shape)

df_jul_2022 = pd.concat([df_jul_1, df_jul_2, df_jul_3], ignore_index=True)
display(df_jul_2022.shape)

df_jul_2022 = df_jul_2022[df_jul_2022['Этаж'] == 1]
df_jul_2022 = df_jul_2022.drop_duplicates()
df_jul_2022 = df_jul_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_jul_2022.shape)

df_jul_2022 = df_jul_2022[(df_jul_2022['Площадь, м2'] < 1000) & (df_jul_2022['Площадь, м2'] > 20)]
display(df_jul_2022.shape)

df_jul_2022 = df_jul_2022.dropna(subset=['Описание'])
condition1 = df_jul_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_jul_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_jul_2022 = df_jul_2022[~condition1 & ~mask]
display(df_jul_2022.shape)

df_jul_2022 = df_jul_2022[~(((df_jul_2022['Тип'] == 'Сдам') & (df_jul_2022['Стоимость, м2'] > 100000)) | ((df_jul_2022['Тип'] == 'Продам') & (df_jul_2022['Стоимость, м2'] > 10000000)))]
display(df_jul_2022.shape)

# Добавляем столбец "Категория Площади"
df_jul_2022['Категория Площади'] = df_jul_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_jul_2022['Район'] = df_jul_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_jul_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_jul_2022.loc[df_jul_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_jul_2022.loc[df_jul_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_jul_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jul_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jul_22_street.xlsx"}')

(10000, 13)

(10000, 13)

(3380, 13)

(23380, 13)

(9496, 13)

(8385, 13)

(8164, 13)

(8137, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jul_22_street.xlsx


## Август 2022

In [15]:
df_aug_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Август_2022/Август_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_aug_1 = df_aug_1.drop_duplicates()
#display(df_aug_1.head(3))
display(df_aug_1.shape)

df_aug_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Август_2022/Август_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_aug_2 = df_aug_2.drop_duplicates()
#display(df_aug_2.head(3))
display(df_aug_2.shape)

df_aug_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Август_2022/Август_2022_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_aug_3 = df_aug_3.drop_duplicates()
#display(df_aug_3.head(3))
display(df_aug_3.shape)

df_aug_2022 = pd.concat([df_aug_1, df_aug_2, df_aug_3], ignore_index=True)
display(df_aug_2022.shape)

df_aug_2022 = df_aug_2022[df_aug_2022['Этаж'] == 1]
df_aug_2022 = df_aug_2022.drop_duplicates()
df_aug_2022 = df_aug_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_aug_2022.shape)

df_aug_2022 = df_aug_2022[(df_aug_2022['Площадь, м2'] < 1000) & (df_aug_2022['Площадь, м2'] > 20)]
display(df_aug_2022.shape)

df_aug_2022 = df_aug_2022.dropna(subset=['Описание'])
condition1 = df_aug_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_aug_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_aug_2022 = df_aug_2022[~condition1 & ~mask]
display(df_aug_2022.shape)

df_aug_2022 = df_aug_2022[~(((df_aug_2022['Тип'] == 'Сдам') & (df_aug_2022['Стоимость, м2'] > 100000)) | ((df_aug_2022['Тип'] == 'Продам') & (df_aug_2022['Стоимость, м2'] > 10000000)))]
display(df_aug_2022.shape)

# Добавляем столбец "Категория Площади"
df_aug_2022['Категория Площади'] = df_aug_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_aug_2022['Район'] = df_aug_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_aug_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_aug_2022.loc[df_aug_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_aug_2022.loc[df_aug_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_aug_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_aug_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_aug_22_street.xlsx"}')

(10000, 13)

(10000, 13)

(1533, 13)

(21533, 13)

(9303, 13)

(8137, 13)

(7959, 13)

(7933, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_aug_22_street.xlsx


## Сентябрь 2022

In [16]:
df_sep_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Сентябрь_2022/Сентябрь_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_sep_1 = df_sep_1.drop_duplicates()
#display(df_sep_1.head(3))
display(df_sep_1.shape)

df_sep_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Сентябрь_2022/Сентябрь_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_sep_2 = df_sep_2.drop_duplicates()
#display(df_sep_2.head(3))
display(df_sep_2.shape)

df_sep_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Сентябрь_2022/Сентябрь_2022_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_sep_3 = df_sep_3.drop_duplicates()
#display(df_sep_3.head(3))
display(df_sep_3.shape)

df_sep_2022 = pd.concat([df_sep_1, df_sep_2, df_sep_3], ignore_index=True)
display(df_sep_2022.shape)

df_sep_2022 = df_sep_2022[df_sep_2022['Этаж'] == 1]
df_sep_2022 = df_sep_2022.drop_duplicates()
df_sep_2022 = df_sep_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_sep_2022.shape)

df_sep_2022 = df_sep_2022[(df_sep_2022['Площадь, м2'] < 1000) & (df_sep_2022['Площадь, м2'] > 20)]
display(df_sep_2022.shape)

df_sep_2022 = df_sep_2022.dropna(subset=['Описание'])
condition1 = df_sep_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_sep_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_sep_2022 = df_sep_2022[~condition1 & ~mask]
display(df_sep_2022.shape)

df_sep_2022 = df_sep_2022[~(((df_sep_2022['Тип'] == 'Сдам') & (df_sep_2022['Стоимость, м2'] > 100000)) | ((df_sep_2022['Тип'] == 'Продам') & (df_sep_2022['Стоимость, м2'] > 10000000)))]
display(df_sep_2022.shape)

# Добавляем столбец "Категория Площади"
df_sep_2022['Категория Площади'] = df_sep_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_sep_2022['Район'] = df_sep_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_sep_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_sep_2022.loc[df_sep_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_sep_2022.loc[df_sep_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_sep_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_sep_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_sep_22_street.xlsx"}')

(10000, 13)

(10000, 13)

(1350, 13)

(21350, 13)

(9396, 13)

(8145, 13)

(7952, 13)

(7936, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_sep_22_street.xlsx


In [28]:
three_m = pd.concat([df_jul_2022, df_aug_2022, df_sep_2022])
three_m.drop_duplicates()
three_m.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first', inplace=True)
display(three_m.shape)

three_m.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/3кв. 2022.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/3кв. 2022.xlsx"}')

(20074, 16)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/3кв. 2022.xlsx


## Октябрь 2022

In [29]:
# df_oct_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2022/Октябрь_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_1 = df_oct_1.drop_duplicates()
# #display(df_oct_1.head(3))
# display(df_oct_1.shape)


# df_oct_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2022/Октябрь_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_2 = df_oct_2.drop_duplicates()
# #display(df_oct_2.head(3))
# display(df_oct_2.shape)

# df_oct_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2022/Октябрь_2022_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_3 = df_oct_3.drop_duplicates()
# #display(df_oct_3.head(3))
# display(df_oct_3.shape)

# df_oct_2022 = pd.concat([df_oct_1, df_oct_2, df_oct_3], ignore_index=True)
# display(df_oct_2022.shape)

# df_oct_2022 = df_oct_2022[df_oct_2022['Этаж'] == 1]
# df_oct_2022 = df_oct_2022.drop_duplicates()
# df_oct_2022 = df_oct_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
# display(df_oct_2022.shape)

# df_oct_2022 = df_oct_2022[(df_oct_2022['Площадь, м2'] < 1000) & (df_oct_2022['Площадь, м2'] > 20)]
# display(df_oct_2022.shape)

# df_oct_2022 = df_oct_2022.dropna(subset=['Описание'])
# condition1 = df_oct_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
# mask = df_oct_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
# df_aug_2023 = df_oct_2022[~condition1 & ~mask]
# display(df_oct_2022.shape)

# df_oct_2022 = df_oct_2022[~(((df_oct_2022['Тип'] == 'Сдам') & (df_oct_2022['Стоимость, м2'] > 100000)) | ((df_oct_2022['Тип'] == 'Продам') & (df_oct_2022['Стоимость, м2'] > 10000000)))]
# display(df_oct_2022.shape)




# # Добавляем столбец "Категория Площади"
# df_oct_2022['Категория Площади'] = df_oct_2022['Площадь, м2'].apply(categorize_area)

# # Добавляем столбец "Район"
# df_oct_2022['Район'] = df_oct_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# # Добавляем столбец "Категория стоимости"
# df_oct_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
# df_oct_2022.loc[df_oct_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_oct_2022.loc[df_oct_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)




# df_oct_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_oct_22_street.xlsx', index=False)
# print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_oct_22_street.xlsx"}')

(10001, 13)

(10001, 13)

(2445, 13)

(22447, 13)

(9673, 13)

(8369, 13)

(8368, 13)

(8344, 13)

## Ноябрь 2022

In [18]:
df_nov_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2022/Ноябрь_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_nov_1 = df_nov_1.drop_duplicates()
#display(df_nov_1.head(3))
display(df_nov_1.shape)

df_nov_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2022/Ноябрь_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_nov_2 = df_nov_2.drop_duplicates()
#display(df_nov_2.head(3))
display(df_nov_2.shape)

df_nov_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2022/Ноябрь_2022_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_nov_3 = df_nov_3.drop_duplicates()
#display(df_nov_3.head(3))
display(df_nov_3.shape)

df_nov_2022 = pd.concat([df_nov_1, df_nov_2, df_nov_3], ignore_index=True)
display(df_nov_2022.shape)

df_nov_2022 = df_nov_2022[df_nov_2022['Этаж'] == 1]
df_nov_2022 = df_nov_2022.drop_duplicates()
df_nov_2022 = df_nov_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_nov_2022.shape)

df_nov_2022 = df_nov_2022[(df_nov_2022['Площадь, м2'] < 1000) & (df_nov_2022['Площадь, м2'] > 20)]
display(df_nov_2022.shape)

df_nov_2022 = df_nov_2022.dropna(subset=['Описание'])
condition1 = df_nov_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_nov_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_nov_2022 = df_nov_2022[~condition1 & ~mask]
display(df_nov_2022.shape)

df_nov_2022 = df_nov_2022[~(((df_nov_2022['Тип'] == 'Сдам') & (df_nov_2022['Стоимость, м2'] > 100000)) | ((df_nov_2022['Тип'] == 'Продам') & (df_nov_2022['Стоимость, м2'] > 10000000)))]
display(df_nov_2022.shape)

# Добавляем столбец "Категория Площади"
df_nov_2022['Категория Площади'] = df_nov_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_nov_2022['Район'] = df_nov_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_nov_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_nov_2022.loc[df_nov_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_nov_2022.loc[df_nov_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_nov_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_22_street.xlsx"}')

(10000, 13)

(10000, 13)

(8798, 13)

(28798, 13)

(12224, 13)

(10805, 13)

(10539, 13)

(10506, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_22_street.xlsx


## Декабрь 2022

In [19]:
df_dec_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Декабрь_2022/Декабрь_2022_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_1 = df_dec_1.drop_duplicates()
#display(df_dec_1.head(3))
display(df_dec_1.shape)

df_dec_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Декабрь_2022/Декабрь_2022_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_2 = df_dec_2.drop_duplicates()
#display(df_dec_2.head(3))
display(df_dec_2.shape)

df_dec_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Декабрь_2022/Декабрь_2022_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_3 = df_dec_3.drop_duplicates()
#display(df_dec_3.head(3))
display(df_dec_3.shape)

df_dec_2022 = pd.concat([df_dec_1, df_dec_2, df_dec_3], ignore_index=True)
display(df_dec_2022.shape)

df_dec_2022 = df_dec_2022[df_dec_2022['Этаж'] == 1]
df_dec_2022 = df_dec_2022.drop_duplicates()
df_dec_2022 = df_dec_2022.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_dec_2022.shape)

df_dec_2022 = df_dec_2022[(df_dec_2022['Площадь, м2'] < 1000) & (df_dec_2022['Площадь, м2'] > 20)]
display(df_dec_2022.shape)

df_dec_2022 = df_dec_2022.dropna(subset=['Описание'])
condition1 = df_dec_2022['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_dec_2022['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_dec_2022 = df_dec_2022[~condition1 & ~mask]
display(df_dec_2022.shape)

df_dec_2022 = df_dec_2022[~(((df_dec_2022['Тип'] == 'Сдам') & (df_dec_2022['Стоимость, м2'] > 100000)) | ((df_dec_2022['Тип'] == 'Продам') & (df_dec_2022['Стоимость, м2'] > 10000000)))]
display(df_dec_2022.shape)

# Добавляем столбец "Категория Площади"
df_dec_2022['Категория Площади'] = df_dec_2022['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_dec_2022['Район'] = df_dec_2022.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_dec_2022['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_dec_2022.loc[df_dec_2022['Тип'] == 'Продам', 'Категория Стоимости'] = df_dec_2022.loc[df_dec_2022['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_dec_2022.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_22_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_22_street.xlsx"}')

(10000, 13)

(10000, 13)

(164, 13)

(20164, 13)

(10505, 13)

(9218, 13)

(8989, 13)

(8940, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_22_street.xlsx


In [30]:
three_m = pd.concat([df_oct_2022, df_nov_2022, df_dec_2022])
three_m.drop_duplicates()
three_m.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first', inplace=True)
display(three_m.shape)

three_m.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/4кв. 2022.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/4кв. 2022.xlsx"}')

(23506, 16)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/4кв. 2022.xlsx


## Январь 2023

In [20]:
df_jan_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2023/Январь_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_1 = df_jan_1.drop_duplicates()
#display(df_jan_1.head(3))
display(df_jan_1.shape)

df_jan_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2023/Январь_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_2 = df_jan_2.drop_duplicates()
#display(df_jan_2.head(3))
display(df_jan_2.shape)

df_jan_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2023/Январь_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_3 = df_jan_3.drop_duplicates()
#display(df_jan_3.head(3))
display(df_jan_3.shape)

df_jan_2023 = pd.concat([df_jan_1, df_jan_2, df_jan_3], ignore_index=True)
display(df_jan_2023.shape)

df_jan_2023 = df_jan_2023[df_jan_2023['Этаж'] == 1]
df_jan_2023 = df_jan_2023.drop_duplicates()
df_jan_2023 = df_jan_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_jan_2023.shape)

df_jan_2023 = df_jan_2023[(df_jan_2023['Площадь, м2'] < 1000) & (df_jan_2023['Площадь, м2'] > 20)]
display(df_jan_2023.shape)

df_jan_2023 = df_jan_2023.dropna(subset=['Описание'])
condition1 = df_jan_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_jan_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_jan_2023 = df_jan_2023[~condition1 & ~mask]
display(df_jan_2023.shape)

df_jan_2023 = df_jan_2023[~(((df_jan_2023['Тип'] == 'Сдам') & (df_jan_2023['Стоимость, м2'] > 100000)) | ((df_jan_2023['Тип'] == 'Продам') & (df_jan_2023['Стоимость, м2'] > 10000000)))]
display(df_jan_2023.shape)

# Добавляем столбец "Категория Площади"
df_jan_2023['Категория Площади'] = df_jan_2023['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_jan_2023['Район'] = df_jan_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_jan_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_jan_2023.loc[df_jan_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_jan_2023.loc[df_jan_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_jan_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_23_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(2084, 13)

(22084, 13)

(12157, 13)

(10379, 13)

(10107, 13)

(10063, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_23_street.xlsx


## Февраль 2023

In [21]:
df_feb_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2023/Февраль_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_1 = df_feb_1.drop_duplicates()
#display(df_feb_1.head(3))
display(df_feb_1.shape)

df_feb_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2023/Февраль_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_2 = df_feb_2.drop_duplicates()
#display(df_feb_2.head(3))
display(df_feb_2.shape)

df_feb_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2023/Февраль_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_3 = df_feb_3.drop_duplicates()
#display(df_feb_3.head(3))
display(df_feb_3.shape)

df_feb_2023 = pd.concat([df_feb_1, df_feb_2, df_feb_3], ignore_index=True)
display(df_feb_2023.shape)

df_feb_2023 = df_feb_2023[df_feb_2023['Этаж'] == 1]
df_feb_2023 = df_feb_2023.drop_duplicates()
df_feb_2023 = df_feb_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_feb_2023.shape)

df_feb_2023 = df_feb_2023[(df_feb_2023['Площадь, м2'] < 1000) & (df_feb_2023['Площадь, м2'] > 20)]
display(df_feb_2023.shape)

df_feb_2023 = df_feb_2023.dropna(subset=['Описание'])
condition1 = df_feb_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_feb_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_feb_2023 = df_feb_2023[~condition1 & ~mask]
display(df_feb_2023.shape)

df_feb_2023 = df_feb_2023[~(((df_feb_2023['Тип'] == 'Сдам') & (df_feb_2023['Стоимость, м2'] > 100000)) | ((df_feb_2023['Тип'] == 'Продам') & (df_feb_2023['Стоимость, м2'] > 10000000)))]
display(df_feb_2023.shape)

# Добавляем столбец "Категория Площади"
df_feb_2023['Категория Площади'] = df_feb_2023['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_feb_2023['Район'] = df_feb_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_feb_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_feb_2023.loc[df_feb_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_feb_2023.loc[df_feb_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_feb_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_23_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(2989, 13)

(22989, 13)

(10734, 13)

(9409, 13)

(9158, 13)

(9120, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_23_street.xlsx


## Март 2023

In [22]:
df_mar_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Март_2023/Март_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_mar_1 = df_mar_1.drop_duplicates()
#display(df_mar_1.head(3))
display(df_mar_1.shape)

df_mar_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Март_2023/Март_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_mar_2 = df_mar_2.drop_duplicates()
#display(df_mar_2.head(3))
display(df_mar_2.shape)

df_mar_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Март_2023/Март_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_mar_3 = df_mar_3.drop_duplicates()
#display(df_mar_3.head(3))
display(df_mar_3.shape)

df_mar_2023 = pd.concat([df_mar_1, df_mar_2, df_mar_3], ignore_index=True)
display(df_mar_2023.shape)

df_mar_2023 = df_mar_2023[df_mar_2023['Этаж'] == 1]
df_mar_2023 = df_mar_2023.drop_duplicates()
df_mar_2023 = df_mar_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_mar_2023.shape)

df_mar_2023 = df_mar_2023[(df_mar_2023['Площадь, м2'] < 1000) & (df_mar_2023['Площадь, м2'] > 20)]
display(df_mar_2023.shape)

df_mar_2023 = df_mar_2023.dropna(subset=['Описание'])
condition1 = df_mar_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_mar_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_mar_2023 = df_mar_2023[~condition1 & ~mask]
display(df_mar_2023.shape)

df_mar_2023 = df_mar_2023[~(((df_mar_2023['Тип'] == 'Сдам') & (df_mar_2023['Стоимость, м2'] > 100000)) | ((df_mar_2023['Тип'] == 'Продам') & (df_mar_2023['Стоимость, м2'] > 10000000)))]
display(df_mar_2023.shape)

# Добавляем столбец "Категория Площади"
df_mar_2023['Категория Площади'] = df_mar_2023['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_mar_2023['Район'] = df_mar_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_mar_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_mar_2023.loc[df_mar_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_mar_2023.loc[df_mar_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_mar_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_23_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(5616, 13)

(25616, 13)

(11946, 13)

(10394, 13)

(10146, 13)

(10084, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_23_street.xlsx


In [31]:
three_m = pd.concat([df_jan_2023, df_feb_2023, df_mar_2023])
three_m.drop_duplicates()
three_m.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first', inplace=True)
display(three_m.shape)

three_m.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/1кв. 2023.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/1кв. 2023.xlsx"}')

(24457, 16)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/1кв. 2023.xlsx


## Апрель 2023

In [23]:
df_apr_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Апрель_2023/Апрель_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_apr_1 = df_apr_1.drop_duplicates()
#display(df_apr_1.head(3))
display(df_apr_1.shape)

df_apr_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Апрель_2023/Апрель_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_apr_2 = df_apr_2.drop_duplicates()
#display(df_apr_2.head(3))
display(df_apr_2.shape)

df_apr_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Апрель_2023/Апрель_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_apr_3 = df_apr_3.drop_duplicates()
#display(df_apr_3.head(3))
display(df_apr_3.shape)

df_apr_2023 = pd.concat([df_apr_1, df_apr_2, df_apr_3], ignore_index=True)
display(df_apr_2023.shape)

df_apr_2023 = df_apr_2023[df_apr_2023['Этаж'] == 1]
df_apr_2023 = df_apr_2023.drop_duplicates()
df_apr_2023 = df_apr_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_apr_2023.shape)

df_apr_2023 = df_apr_2023[(df_apr_2023['Площадь, м2'] < 1000) & (df_apr_2023['Площадь, м2'] > 20)]
display(df_apr_2023.shape)

df_apr_2023 = df_apr_2023.dropna(subset=['Описание'])
condition1 = df_apr_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_apr_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_apr_2023 = df_apr_2023[~condition1 & ~mask]
display(df_apr_2023.shape)

df_apr_2023 = df_apr_2023[~(((df_apr_2023['Тип'] == 'Сдам') & (df_apr_2023['Стоимость, м2'] > 100000)) | ((df_apr_2023['Тип'] == 'Продам') & (df_apr_2023['Стоимость, м2'] > 10000000)))]
display(df_apr_2023.shape)

# Добавляем столбец "Категория Площади"
df_apr_2023['Категория Площади'] = df_apr_2023['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_apr_2023['Район'] = df_apr_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_apr_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_apr_2023.loc[df_apr_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_apr_2023.loc[df_apr_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_apr_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_23_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(2775, 13)

(22775, 13)

(13101, 13)

(11299, 13)

(10978, 13)

(10933, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_23_street.xlsx


## Май 2023

In [24]:
df_may_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Май_2023/Май_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_may_1 = df_may_1.drop_duplicates()
#display(df_may_1.head(3))
display(df_may_1.shape)

df_may_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Май_2023/Май_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_may_2 = df_may_2.drop_duplicates()
#display(df_may_2.head(3))
display(df_may_2.shape)

df_may_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Май_2023/Май_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_may_3 = df_may_3.drop_duplicates()
#display(df_may_3.head(3))
display(df_may_3.shape)

df_may_2023 = pd.concat([df_may_1, df_may_2, df_may_3], ignore_index=True)
display(df_may_2023.shape)

df_may_2023 = df_may_2023[df_may_2023['Этаж'] == 1]
df_may_2023 = df_may_2023.drop_duplicates()
df_may_2023 = df_may_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_may_2023.shape)

df_may_2023 = df_may_2023[(df_may_2023['Площадь, м2'] < 1000) & (df_may_2023['Площадь, м2'] > 20)]
display(df_may_2023.shape)

df_may_2023 = df_may_2023.dropna(subset=['Описание'])
condition1 = df_may_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_may_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
df_may_2023 = df_may_2023[~condition1 & ~mask]
display(df_may_2023.shape)

df_may_2023 = df_may_2023[~(((df_may_2023['Тип'] == 'Сдам') & (df_may_2023['Стоимость, м2'] > 100000)) | ((df_may_2023['Тип'] == 'Продам') & (df_may_2023['Стоимость, м2'] > 10000000)))]
display(df_may_2023.shape)

# Добавляем столбец "Категория Площади"
df_may_2023['Категория Площади'] = df_may_2023['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_may_2023['Район'] = df_may_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_may_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_may_2023.loc[df_may_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_may_2023.loc[df_may_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)

df_may_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_23_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(1200, 13)

(21200, 13)

(12697, 13)

(10924, 13)

(10655, 13)

(10640, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_23_street.xlsx


## Июнь 2023

In [32]:
# df_june_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Июнь_2023/Июнь.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_june_1 = df_june_1.drop_duplicates()
# #display(df_june_1.head(3))
# display(df_june_1.shape)


# df_june_2023 = df_june_1

# df_june_2023 = df_june_2023[df_june_2023['Этаж'] == 1]
# df_june_2023 = df_june_2023.drop_duplicates()
# df_june_2023 = df_june_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
# display(df_june_2023.shape)

# df_june_2023 = df_june_2023[(df_june_2023['Площадь, м2'] < 1000) & (df_june_2023['Площадь, м2'] > 20)]
# display(df_june_2023.shape)

# df_june_2023 = df_june_2023.dropna(subset=['Описание'])
# condition1 = df_june_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
# mask = df_june_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
# df_june_2023 = df_june_2023[~condition1 & ~mask]
# display(df_june_2023.shape)

# df_june_2023 = df_june_2023[~(((df_june_2023['Тип'] == 'Сдам') & (df_june_2023['Стоимость, м2'] > 100000)) | ((df_june_2023['Тип'] == 'Продам') & (df_june_2023['Стоимость, м2'] > 10000000)))]
# display(df_june_2023.shape)


# # Добавляем столбец "Категория Площади"
# df_june_2023['Категория Площади'] = df_june_2023['Площадь, м2'].apply(categorize_area)

# # Добавляем столбец "Район"
# df_june_2023['Район'] = df_june_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# # Добавляем столбец "Категория стоимости"
# df_june_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
# df_june_2023.loc[df_june_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_june_2023.loc[df_june_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



# df_june_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_june_23_street.xlsx', index=False)
# print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_june_23_street.xlsx"}')

(27912, 13)

(15651, 13)

(13505, 13)

(13183, 13)

(13145, 13)

In [33]:
three_m = pd.concat([df_apr_2023, df_may_2023, df_june_2023])
three_m.drop_duplicates()
three_m.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first', inplace=True)
display(three_m.shape)

three_m.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/2кв. 2023.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/2кв. 2023.xlsx"}')

(29688, 16)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/2кв. 2023.xlsx


## Июль 2023

In [7]:
# df_july_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Июль_2023/Июль.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_july_1 = df_july_1.drop_duplicates()
# #display(df_july_1.head(3))
# display(df_july_1.shape)


# df_july_2023 = df_july_1

# df_july_2023 = df_july_2023[df_july_2023['Этаж'] == 1]
# df_july_2023 = df_july_2023.drop_duplicates()
# df_july_2023 = df_july_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
# display(df_july_2023.shape)

# df_july_2023 = df_july_2023[(df_july_2023['Площадь, м2'] < 1000) & (df_july_2023['Площадь, м2'] > 20)]
# display(df_july_2023.shape)

# df_july_2023 = df_july_2023.dropna(subset=['Описание'])
# condition1 = df_july_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
# mask = df_july_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
# condition1 = condition1.fillna(False)
# mask = mask.fillna(False)
# df_july_2023 = df_july_2023[~condition1 & ~mask]
# display(df_july_2023.shape)

# df_july_2023 = df_july_2023[~(((df_july_2023['Тип'] == 'Сдам') & (df_july_2023['Стоимость, м2'] > 100000)) | ((df_july_2023['Тип'] == 'Продам') & (df_july_2023['Стоимость, м2'] > 10000000)))]
# display(df_july_2023.shape)



# # Добавляем столбец "Категория Площади"
# df_july_2023['Категория Площади'] = df_july_2023['Площадь, м2'].apply(categorize_area)

# # Добавляем столбец "Район"
# df_july_2023['Район'] = df_july_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# # Добавляем столбец "Категория стоимости"
# df_july_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
# df_july_2023.loc[df_july_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_july_2023.loc[df_july_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



# df_july_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_july_23_street.xlsx', index=False)
# print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_july_23_street.xlsx"}')

(44885, 13)

(26428, 13)

(22436, 13)

(21745, 13)

(21707, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_july_23_street.xlsx


## Август 2023

In [8]:
# df_aug_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Август_2023/Август_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_aug_1 = df_aug_1.drop_duplicates()
# #display(df_aug_1.head(3))
# display(df_aug_1.shape)


# df_aug_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Август_2023/Август_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_aug_2 = df_aug_2.drop_duplicates()
# #display(df_aug_2.head(3))
# display(df_aug_2.shape)

# df_aug_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Август_2023/Август_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_aug_3 = df_aug_3.drop_duplicates()
# #display(df_aug_3.head(3))
# display(df_aug_3.shape)

# df_aug_2023 = pd.concat([df_aug_1, df_aug_2, df_aug_3], ignore_index=True)
# display(df_aug_2023.shape)

# df_aug_2023 = df_aug_2023[df_aug_2023['Этаж'] == 1]
# df_aug_2023 = df_aug_2023.drop_duplicates()
# df_aug_2023 = df_aug_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
# display(df_aug_2023.shape)

# df_aug_2023 = df_aug_2023[(df_aug_2023['Площадь, м2'] < 1000) & (df_aug_2023['Площадь, м2'] > 20)]
# display(df_aug_2023.shape)

# df_aug_2023 = df_aug_2023.dropna(subset=['Описание'])
# condition1 = df_aug_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
# mask = df_aug_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
# df_aug_2023 = df_aug_2023[~condition1 & ~mask]
# display(df_aug_2023.shape)

# df_aug_2023 = df_aug_2023[~(((df_aug_2023['Тип'] == 'Сдам') & (df_aug_2023['Стоимость, м2'] > 100000)) | ((df_aug_2023['Тип'] == 'Продам') & (df_aug_2023['Стоимость, м2'] > 10000000)))]
# display(df_aug_2023.shape)



# # Добавляем столбец "Категория Площади"
# df_aug_2023['Категория Площади'] = df_aug_2023['Площадь, м2'].apply(categorize_area)

# # Добавляем столбец "Район"
# df_aug_2023['Район'] = df_aug_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# # Добавляем столбец "Категория стоимости"
# df_aug_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
# df_aug_2023.loc[df_aug_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_aug_2023.loc[df_aug_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



# df_aug_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_aug_23_street.xlsx', index=False)
# print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_aug_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(2274, 13)

(22274, 13)

(13831, 13)

(11818, 13)

(11459, 13)

(11428, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_aug_23_street.xlsx


## Сентябрь 2023

In [9]:
# df_sep_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Сентябрь_2023/Сентябрь_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_sep_1 = df_sep_1.drop_duplicates()
# #display(df_sep_1.head(3))
# display(df_sep_1.shape)


# df_sep_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Сентябрь_2023/Сентябрь_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_sep_2 = df_sep_2.drop_duplicates()
# #display(df_sep_2.head(3))
# display(df_sep_2.shape)

# df_sep_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Сентябрь_2023/Сентябрь_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_sep_3 = df_sep_3.drop_duplicates()
# #display(df_sep_3.head(3))
# display(df_sep_3.shape)

# df_sep_4 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Сентябрь_2023/Сентябрь_2023_4.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_sep_4 = df_sep_4.drop_duplicates()
# #display(df_sep_4.head(3))
# display(df_sep_4.shape)

# df_sep_2023 = pd.concat([df_sep_1, df_sep_2, df_sep_3, df_sep_4], ignore_index=True)
# display(df_sep_2023.shape)

# df_sep_2023 = df_sep_2023[df_sep_2023['Этаж'] == 1]
# df_sep_2023 = df_sep_2023.drop_duplicates()
# df_sep_2023 = df_sep_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
# display(df_sep_2023.shape)

# df_sep_2023 = df_sep_2023[(df_sep_2023['Площадь, м2'] < 1000) & (df_sep_2023['Площадь, м2'] > 20)]
# display(df_sep_2023.shape)

# df_sep_2023 = df_sep_2023.dropna(subset=['Описание'])
# condition1 = df_sep_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
# mask = df_sep_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
# df_sep_2023 = df_sep_2023[~condition1 & ~mask]
# display(df_sep_2023.shape)

# df_sep_2023 = df_sep_2023[~(((df_sep_2023['Тип'] == 'Сдам') & (df_sep_2023['Стоимость, м2'] > 100000)) | ((df_sep_2023['Тип'] == 'Продам') & (df_sep_2023['Стоимость, м2'] > 10000000)))]
# display(df_sep_2023.shape)



# # Добавляем столбец "Категория Площади"
# df_sep_2023['Категория Площади'] = df_sep_2023['Площадь, м2'].apply(categorize_area)

# # Добавляем столбец "Район"
# df_sep_2023['Район'] = df_sep_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# # Добавляем столбец "Категория стоимости"
# df_sep_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
# .loc[df_sep_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_sep_2023.loc[df_sep_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



# df_sep_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_sep_23_street.xlsx', index=False)
# print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_sep_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(10000, 13)

(6517, 13)

(36517, 13)

(22216, 13)

(18757, 13)

(18203, 13)

(18164, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_sep_23_street.xlsx


In [ ]:
three_m = pd.concat([df_july_2023, df_aug_2023, df_sep_2023])
three_m.drop_duplicates()
three_m.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first', inplace=True)
display(three_m.shape)

three_m.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/3кв. 2023.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/Кварталы/3кв. 2023.xlsx"}')

## Октябрь 2023

In [10]:
# df_oct_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2023/Октябрь_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_1 = df_oct_1.drop_duplicates()
# #display(df_oct_1.head(3))
# display(df_oct_1.shape)

# df_oct_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2023/Октябрь_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_2 = df_oct_2.drop_duplicates()
# #display(df_oct_2.head(3))
# display(df_oct_2.shape)

# df_oct_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2023/Октябрь_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_3 = df_oct_3.drop_duplicates()
# #display(df_oct_3.head(3))
# display(df_oct_3.shape)

# df_oct_4 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2023/Октябрь_2023_4.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_4 = df_oct_4.drop_duplicates()
# #display(df_oct_4.head(3))
# display(df_oct_4.shape)

# df_oct_5 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2023/Октябрь_2023_5.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_5 = df_oct_5.drop_duplicates()
# #display(df_oct_5.head(3))
# display(df_oct_5.shape)

# df_oct_6 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Октябрь_2023/Октябрь_2023_6.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_oct_6 = df_oct_6.drop_duplicates()
# #display(df_oct_6.head(3))
# display(df_oct_6.shape)


# df_oct_2023 = pd.concat([df_oct_1, df_oct_2, df_oct_3, df_oct_4, df_oct_5, df_oct_6], ignore_index=True)
# display(df_oct_2023.shape)

# df_oct_2023 = df_oct_2023[df_oct_2023['Этаж'] == 1]
# df_oct_2023 = df_oct_2023.drop_duplicates()
# df_oct_2023 = df_oct_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
# display(df_oct_2023.shape)

# df_oct_2023 = df_oct_2023[(df_oct_2023['Площадь, м2'] < 1000) & (df_oct_2023['Площадь, м2'] > 20)]
# display(df_oct_2023.shape)

# df_oct_2023 = df_oct_2023.dropna(subset=['Описание'])
# condition1 = df_oct_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
# mask = df_oct_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
# condition1 = condition1.fillna(False)
# mask = mask.fillna(False)
# df_oct_2023 = df_oct_2023[~condition1 & ~mask]
# display(df_oct_2023.shape)

# df_oct_2023 = df_oct_2023[~(((df_oct_2023['Тип'] == 'Сдам') & (df_oct_2023['Стоимость, м2'] > 100000)) | ((df_oct_2023['Тип'] == 'Продам') & (df_oct_2023['Стоимость, м2'] > 10000000)))]
# display(df_oct_2023.shape)



# # Добавляем столбец "Категория Площади"
# df_oct_2023['Категория Площади'] = df_oct_2023['Площадь, м2'].apply(categorize_area)

# # Добавляем столбец "Район"
# df_oct_2023['Район'] = df_oct_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# # Добавляем столбец "Категория стоимости"
# df_oct_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
# df_oct_2023.loc[df_oct_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_oct_2023.loc[df_oct_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



# df_oct_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_oct_23_street.xlsx', index=False)
# print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_oct_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(10000, 13)

(10000, 13)

(10000, 13)

...

(30341, 13)

(25576, 13)

(24893, 13)

(24852, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_oct_23_street.xlsx


## Ноябрь 2023

In [6]:
# df_nov_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2023/Ноябрь_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_nov_1 = df_nov_1.drop_duplicates()
# #display(df_oct_1.head(3))
# display(df_nov_1.shape)

# df_nov_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2023/Ноябрь_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_nov_2 = df_nov_2.drop_duplicates()
# #display(df_oct_2.head(3))
# display(df_nov_2.shape)

# df_nov_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2023/Ноябрь_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_nov_3 = df_nov_3.drop_duplicates()
# #display(df_oct_3.head(3))
# display(df_nov_3.shape)

# df_nov_4 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2023/Ноябрь_2023_4.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_nov_4 = df_nov_4.drop_duplicates()
# #display(df_oct_4.head(3))
# display(df_nov_4.shape)

# df_nov_5 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Ноябрь_2023/Ноябрь_2023_5.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Стоимость, м2', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
# df_nov_5 = df_nov_5.drop_duplicates()
# #display(df_oct_5.head(3))
# display(df_nov_5.shape)


# df_nov_2023 = pd.concat([df_nov_1, df_nov_2, df_nov_3, df_nov_4, df_nov_5], ignore_index=True)
# display(df_nov_2023.shape)

# df_nov_2023 = df_nov_2023[df_nov_2023['Этаж'] == 1]
# df_nov_2023 = df_nov_2023.drop_duplicates()
# df_nov_2023 = df_nov_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
# display(df_nov_2023.shape)

# df_nov_2023 = df_nov_2023[(df_nov_2023['Площадь, м2'] < 1000) & (df_nov_2023['Площадь, м2'] > 20)]
# display(df_nov_2023.shape)

# df_nov_2023 = df_nov_2023.dropna(subset=['Описание'])
# condition1 = df_nov_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
# mask = df_nov_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
# condition1 = condition1.fillna(False)
# mask = mask.fillna(False)
# df_nov_2023 = df_nov_2023[~condition1 & ~mask]
# display(df_nov_2023.shape)

# df_nov_2023 = df_nov_2023[~(((df_nov_2023['Тип'] == 'Сдам') & (df_nov_2023['Стоимость, м2'] > 100000)) | ((df_nov_2023['Тип'] == 'Продам') & (df_nov_2023['Стоимость, м2'] > 10000000)))]
# display(df_nov_2023.shape)



# # Добавляем столбец "Категория Площади"
# df_nov_2023['Категория Площади'] = df_nov_2023['Площадь, м2'].apply(categorize_area)

# # Добавляем столбец "Район"
# df_nov_2023['Район'] = df_nov_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# # Добавляем столбец "Категория стоимости"
# df_nov_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
# df_nov_2023.loc[df_nov_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_nov_2023.loc[df_nov_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



# df_nov_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_23_street.xlsx', index=False)
# print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(10000, 13)

(10000, 13)

(9538, 13)

...

(29239, 13)

(24635, 13)

(23987, 13)

(23937, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_nov_23_street.xlsx


## Декабрь 2023

In [6]:
df_dec_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Декабрь_2023/Декабрь_2023_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_1['Стоимость, м2'] = df_dec_1['Стоимость'] / df_dec_1['Площадь, м2']
df_dec_1 = df_dec_1.drop_duplicates()
#display(df_oct_1.head(3))
display(df_dec_1.shape)

df_dec_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Декабрь_2023/Декабрь_2023_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_2['Стоимость, м2'] = df_dec_2['Стоимость'] / df_dec_2['Площадь, м2']
df_dec_2 = df_dec_2.drop_duplicates()
#display(df_oct_2.head(3))
display(df_dec_2.shape)

df_dec_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Декабрь_2023/Декабрь_2023_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_3['Стоимость, м2'] = df_dec_3['Стоимость'] / df_dec_3['Площадь, м2']
df_dec_3 = df_dec_3.drop_duplicates()
#display(df_oct_3.head(3))
display(df_dec_3.shape)



df_dec_2023 = pd.concat([df_dec_1, df_dec_2, df_dec_3], ignore_index=True)
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[df_dec_2023['Этаж'] == 1]
df_dec_2023 = df_dec_2023.drop_duplicates()
df_dec_2023 = df_dec_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[(df_dec_2023['Площадь, м2'] < 1000) & (df_dec_2023['Площадь, м2'] > 20)]
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023.dropna(subset=['Описание'])
condition1 = df_dec_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_dec_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
condition1 = condition1.fillna(False)
mask = mask.fillna(False)
df_dec_2023 = df_dec_2023[~condition1 & ~mask]
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[~(((df_dec_2023['Тип'] == 'Сдам') & (df_dec_2023['Стоимость, м2'] > 100000)) | ((df_dec_2023['Тип'] == 'Продам') & (df_dec_2023['Стоимость, м2'] > 10000000)))]
display(df_dec_2023.shape)



# Добавляем столбец "Категория Площади"
df_dec_2023['Категория Площади'] = df_dec_2023['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_dec_2023['Район'] = df_dec_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_dec_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_dec_2023.loc[df_dec_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_dec_2023.loc[df_dec_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



df_dec_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_23_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_23_street.xlsx"}')

(10000, 13)

(10000, 13)

(926, 13)

(20926, 13)

(13250, 13)

(11346, 13)

(11071, 13)

(11048, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_dec_23_street.xlsx


## Январь 2024

In [8]:
df_jan_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2024/Январь_2024_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_1['Стоимость, м2'] = df_jan_1['Стоимость'] / df_jan_1['Площадь, м2']
df_jan_1 = df_jan_1.drop_duplicates()
#display(df_oct_1.head(3))
display(df_jan_1.shape)

df_jan_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2024/Январь_2024_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_2['Стоимость, м2'] = df_jan_2['Стоимость'] / df_jan_2['Площадь, м2']
df_jan_2 = df_jan_2.drop_duplicates()
#display(df_oct_2.head(3))
display(df_jan_2.shape)

df_jan_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2024/Январь_2024_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_3['Стоимость, м2'] = df_jan_3['Стоимость'] / df_jan_3['Площадь, м2']
df_jan_3 = df_jan_3.drop_duplicates()
#display(df_oct_3.head(3))
display(df_jan_3.shape)

df_jan_4 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Январь_2024/Январь_2024_4.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_jan_4['Стоимость, м2'] = df_jan_4['Стоимость'] / df_jan_4['Площадь, м2']
df_jan_4 = df_jan_4.drop_duplicates()
#display(df_oct_4.head(3))
display(df_jan_4.shape)


df_jan_2024 = pd.concat([df_jan_1, df_jan_2, df_jan_3, df_jan_4], ignore_index=True)
display(df_jan_2024.shape)

df_jan_2024 = df_jan_2024[df_jan_2024['Этаж'] == 1]
df_jan_2024 = df_jan_2024.drop_duplicates()
df_jan_2024 = df_jan_2024.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_jan_2024.shape)

df_jan_2024 = df_jan_2024[(df_jan_2024['Площадь, м2'] < 1000) & (df_jan_2024['Площадь, м2'] > 20)]
display(df_jan_2024.shape)

df_jan_2024 = df_jan_2024.dropna(subset=['Описание'])
condition1 = df_jan_2024['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_jan_2024['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
condition1 = condition1.fillna(False)
mask = mask.fillna(False)
df_jan_2024 = df_jan_2024[~condition1 & ~mask]
display(df_jan_2024.shape)

df_jan_2024 = df_jan_2024[~(((df_jan_2024['Тип'] == 'Сдам') & (df_jan_2024['Стоимость, м2'] > 100000)) | ((df_jan_2024['Тип'] == 'Продам') & (df_jan_2024['Стоимость, м2'] > 10000000)))]
display(df_jan_2024.shape)



# Добавляем столбец "Категория Площади"
df_jan_2024['Категория Площади'] = df_jan_2024['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_jan_2024['Район'] = df_jan_2024.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_jan_2024['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_jan_2024.loc[df_jan_2024['Тип'] == 'Продам', 'Категория Стоимости'] = df_jan_2024.loc[df_jan_2024['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



df_jan_2024.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_24_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_24_street.xlsx"}')

(10000, 13)

(10000, 13)

(10000, 13)

(57, 13)

(30057, 13)

(18349, 13)

(15561, 13)

(15170, 13)

(15139, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_jan_24_street.xlsx


## Февраль 2024

In [9]:
df_feb_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2024/Февраль_2024_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_1['Стоимость, м2'] = df_feb_1['Стоимость'] / df_feb_1['Площадь, м2']
df_feb_1 = df_feb_1.drop_duplicates()
#display(df_oct_1.head(3))
display(df_feb_1.shape)

df_feb_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2024/Февраль_2024_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_2['Стоимость, м2'] = df_feb_2['Стоимость'] / df_feb_2['Площадь, м2']
df_feb_2 = df_feb_2.drop_duplicates()
#display(df_oct_2.head(3))
display(df_feb_2.shape)

df_feb_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2024/Февраль_2024_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_3['Стоимость, м2'] = df_feb_3['Стоимость'] / df_feb_3['Площадь, м2']
df_feb_3 = df_feb_3.drop_duplicates()
#display(df_oct_3.head(3))
display(df_feb_3.shape)

df_feb_4 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Февраль_2024/Февраль_2024_4.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_feb_4['Стоимость, м2'] = df_feb_4['Стоимость'] / df_feb_4['Площадь, м2']
df_feb_4 = df_feb_4.drop_duplicates()
#display(df_oct_4.head(3))
display(df_feb_4.shape)


df_feb_2024 = pd.concat([df_feb_1, df_feb_2, df_feb_3, df_feb_4], ignore_index=True)
display(df_feb_2024.shape)

df_feb_2024 = df_feb_2024[df_feb_2024['Этаж'] == 1]
df_feb_2024 = df_feb_2024.drop_duplicates()
df_feb_2024 = df_feb_2024.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_feb_2024.shape)

df_feb_2024 = df_feb_2024[(df_feb_2024['Площадь, м2'] < 1000) & (df_feb_2024['Площадь, м2'] > 20)]
display(df_feb_2024.shape)

df_feb_2024 = df_feb_2024.dropna(subset=['Описание'])
condition1 = df_feb_2024['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_feb_2024['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
condition1 = condition1.fillna(False)
mask = mask.fillna(False)
df_feb_2024 = df_feb_2024[~condition1 & ~mask]
display(df_feb_2024.shape)

df_feb_2024 = df_feb_2024[~(((df_feb_2024['Тип'] == 'Сдам') & (df_feb_2024['Стоимость, м2'] > 100000)) | ((df_feb_2024['Тип'] == 'Продам') & (df_feb_2024['Стоимость, м2'] > 10000000)))]
display(df_feb_2024.shape)



# Добавляем столбец "Категория Площади"
df_feb_2024['Категория Площади'] = df_feb_2024['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_feb_2024['Район'] = df_feb_2024.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_feb_2024['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_feb_2024.loc[df_feb_2024['Тип'] == 'Продам', 'Категория Стоимости'] = df_feb_2024.loc[df_feb_2024['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



df_feb_2024.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_24_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_24_street.xlsx"}')

(10000, 13)

(10000, 13)

(10000, 13)

(6737, 13)

(36737, 13)

(19994, 13)

(16757, 13)

(16307, 13)

(16261, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_feb_24_street.xlsx


## Март 2024

In [11]:
df_mar_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Март_2024/Март_2024_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_mar_1['Стоимость, м2'] = df_mar_1['Стоимость'] / df_mar_1['Площадь, м2']
df_mar_1 = df_mar_1.drop_duplicates()
#display(df_oct_1.head(3))
display(df_mar_1.shape)

df_mar_2024 = df_mar_1

df_mar_2024 = df_mar_2024[df_mar_2024['Этаж'] == 1]
df_mar_2024 = df_mar_2024.drop_duplicates()
df_mar_2024 = df_mar_2024.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_mar_2024.shape)

df_mar_2024 = df_mar_2024[(df_mar_2024['Площадь, м2'] < 1000) & (df_mar_2024['Площадь, м2'] > 20)]
display(df_mar_2024.shape)

df_mar_2024 = df_mar_2024.dropna(subset=['Описание'])
condition1 = df_mar_2024['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_mar_2024['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
condition1 = condition1.fillna(False)
mask = mask.fillna(False)
df_mar_2024 = df_mar_2024[~condition1 & ~mask]
display(df_mar_2024.shape)

df_mar_2024 = df_mar_2024[~(((df_mar_2024['Тип'] == 'Сдам') & (df_mar_2024['Стоимость, м2'] > 100000)) | ((df_mar_2024['Тип'] == 'Продам') & (df_mar_2024['Стоимость, м2'] > 10000000)))]
display(df_mar_2024.shape)



# Добавляем столбец "Категория Площади"
df_mar_2024['Категория Площади'] = df_mar_2024['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_mar_2024['Район'] = df_mar_2024.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_mar_2024['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_mar_2024.loc[df_mar_2024['Тип'] == 'Продам', 'Категория Стоимости'] = df_mar_2024.loc[df_mar_2024['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



df_mar_2024.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_2024_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_2024_street.xlsx"}')

(46119, 13)

(24911, 13)

(20834, 13)

(20382, 13)

(20328, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_mar_2024_street.xlsx


## Апрель 2024

In [5]:
df_dec_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Апрель_2024/Апрель_2024_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_1['Стоимость, м2'] = df_dec_1['Стоимость'] / df_dec_1['Площадь, м2']
df_dec_1 = df_dec_1.drop_duplicates()
#display(df_oct_1.head(3))
display(df_dec_1.shape)

df_dec_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Апрель_2024/Апрель_2024_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_2['Стоимость, м2'] = df_dec_2['Стоимость'] / df_dec_2['Площадь, м2']
df_dec_2 = df_dec_2.drop_duplicates()
#display(df_oct_2.head(3))
display(df_dec_2.shape)

df_dec_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Апрель_2024/Апрель_2024_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_3['Стоимость, м2'] = df_dec_3['Стоимость'] / df_dec_3['Площадь, м2']
df_dec_3 = df_dec_3.drop_duplicates()
#display(df_oct_3.head(3))
display(df_dec_3.shape)



df_dec_2023 = pd.concat([df_dec_1, df_dec_2, df_dec_3], ignore_index=True)
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[df_dec_2023['Этаж'] == 1]
df_dec_2023 = df_dec_2023.drop_duplicates()
df_dec_2023 = df_dec_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[(df_dec_2023['Площадь, м2'] < 1000) & (df_dec_2023['Площадь, м2'] > 20)]
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023.dropna(subset=['Описание'])
condition1 = df_dec_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_dec_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
condition1 = condition1.fillna(False)
mask = mask.fillna(False)
df_dec_2023 = df_dec_2023[~condition1 & ~mask]
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[~(((df_dec_2023['Тип'] == 'Сдам') & (df_dec_2023['Стоимость, м2'] > 100000)) | ((df_dec_2023['Тип'] == 'Продам') & (df_dec_2023['Стоимость, м2'] > 10000000)))]
display(df_dec_2023.shape)



# Добавляем столбец "Категория Площади"
df_dec_2023['Категория Площади'] = df_dec_2023['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_dec_2023['Район'] = df_dec_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_dec_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_dec_2023.loc[df_dec_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_dec_2023.loc[df_dec_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



df_dec_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_24_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_24_street.xlsx"}')

(10000, 13)

(10000, 13)

(9001, 13)

(29001, 13)

(16113, 13)

(13694, 13)

(13404, 13)

(13367, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_apr_24_street.xlsx


## Май 2024

In [6]:
df_dec_1 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Май_2024/Май_2024_1.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_1['Стоимость, м2'] = df_dec_1['Стоимость'] / df_dec_1['Площадь, м2']
df_dec_1 = df_dec_1.drop_duplicates()
#display(df_oct_1.head(3))
display(df_dec_1.shape)

df_dec_2 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Май_2024/Май_2024_2.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_2['Стоимость, м2'] = df_dec_2['Стоимость'] / df_dec_2['Площадь, м2']
df_dec_2 = df_dec_2.drop_duplicates()
#display(df_oct_2.head(3))
display(df_dec_2.shape)

df_dec_3 = pd.read_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Данные/Май_2024/Май_2024_3.xlsx', sheet_name='Worksheet')[['№', 'Категория', 'Тип', 'Этаж', 'Площадь, м2', 'Стоимость', 'Имя контакта', 'Телефон', 'Широта', 'Долгота', 'Описание', 'Ссылка']]
df_dec_3['Стоимость, м2'] = df_dec_3['Стоимость'] / df_dec_3['Площадь, м2']
df_dec_3 = df_dec_3.drop_duplicates()
#display(df_oct_3.head(3))
display(df_dec_3.shape)



df_dec_2023 = pd.concat([df_dec_1, df_dec_2, df_dec_3], ignore_index=True)
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[df_dec_2023['Этаж'] == 1]
df_dec_2023 = df_dec_2023.drop_duplicates()
df_dec_2023 = df_dec_2023.drop_duplicates(subset=['Тип', 'Площадь, м2', 'Широта', 'Долгота'], keep='first')
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[(df_dec_2023['Площадь, м2'] < 1000) & (df_dec_2023['Площадь, м2'] > 20)]
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023.dropna(subset=['Описание'])
condition1 = df_dec_2023['Описание'].str.contains('в ТЦ|в тц|в ТРЦ|в торговом центре|на первом этаже торгового центра|в ТД|в ТРК|в новом ТРК|в составе ТРЦ|на первом торговом уровне ТЦ|в новом ТЦ|ТЦ "Ясенево"|в районном Торговом центре|в современном ТРЦ|на 1 этаже ТЦ|на 1-ом этаже ТЦ|на 1 этаже торгового центра|на 1-м этаже торгового центра|на 1-ом этаже торгового центра', case=False)
mask = df_dec_2023['Описание'].str[:70].str.contains('Торговый центр|Торговый Центр|торговый центр', case=False)
condition1 = condition1.fillna(False)
mask = mask.fillna(False)
df_dec_2023 = df_dec_2023[~condition1 & ~mask]
display(df_dec_2023.shape)

df_dec_2023 = df_dec_2023[~(((df_dec_2023['Тип'] == 'Сдам') & (df_dec_2023['Стоимость, м2'] > 100000)) | ((df_dec_2023['Тип'] == 'Продам') & (df_dec_2023['Стоимость, м2'] > 10000000)))]
display(df_dec_2023.shape)



# Добавляем столбец "Категория Площади"
df_dec_2023['Категория Площади'] = df_dec_2023['Площадь, м2'].apply(categorize_area)

# Добавляем столбец "Район"
df_dec_2023['Район'] = df_dec_2023.apply(lambda row: get_district(Point(row['Долгота'], row['Широта'])), axis=1)

# Добавляем столбец "Категория стоимости"
df_dec_2023['Категория Стоимости'] = "Не определено"  # Значение по умолчанию
df_dec_2023.loc[df_dec_2023['Тип'] == 'Продам', 'Категория Стоимости'] = df_dec_2023.loc[df_dec_2023['Тип'] == 'Продам', 'Стоимость'].apply(categorize_price)



df_dec_2023.to_excel('C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_24_street.xlsx', index=False)
print(f'DataFrame сохранен в {"C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_24_street.xlsx"}')

(20999, 13)

(20000, 13)

(11422, 13)

(52421, 13)

(24071, 13)

(20574, 13)

(20140, 13)

(20071, 13)

DataFrame сохранен в C:/Users/malsh/Desktop/Магазин Магазинов/Готовые df street/df_may_24_street.xlsx


# Конец